# **Lab 2 Aruco markers** 

**General Goal** 

The goal of this project is to introduce how to use aruco markers which is a part of the OpenCV library. Students are also required to work with cameras connected to a system via USB.

**Prerequisite** 

Each exercise is connected to Lab 1 and a lot of code from this project will be reused. In addition, you will need a web-camera or a video feed and a print-out of an aruco marker that is included in the files for this project.

*Tip: It is not mandatory, but it is good if you have a white background when doing this lab.  From OpenCV you can find the following new commands useful to be able to finish the exercises:* 

- Functions:

    ocv2.VideoCapture

    ocv2.adaptiveThreshold

    oaruco.DetectorParameters_create

    ocv2.aruco.Dictionary_get

    oaruco.detectMarkers


## **Step 1 – Preparation of code from first lab** 

Create a script called MyDetectionMethods.py and create a class called MyDetectionMethods. Inside this class create methods that accept image data as input and returns contours. This method should contain the image preprocessing steps done in exercise 6 in the first project. Make one method for canny filter and one for binarization.

In [1]:
import cv2 as cv
import numpy as np

class MyDetectionMethods:

    @staticmethod
    def get_contours_canny(img):
        """
        Accepts a BGR image and returns contours after:
        - convert to grayscale
        - Gaussian blur
        - Canny edge detection
        - morphological closing (optional)
        """

        # Convert to gray
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # Blur to reduce noise
        blurred = cv.GaussianBlur(gray, (5, 5), 0)

        # Canny edge detection
        edges = cv.Canny(blurred, 50, 150)

        # Optional: close gaps in edges
        kernel = np.ones((3, 3), np.uint8)
        edges_closed = cv.morphologyEx(edges, cv.MORPH_CLOSE, kernel)

        # Find contours
        contours, _ = cv.findContours(edges_closed, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

        return contours



    @staticmethod
    def get_contours_binarization(img):
        """
        Accepts a BGR image and returns contours after:
        - convert to grayscale
        - Gaussian blur
        - threshold (binarization)
        """

        # Convert to gray
        gray = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

        # Blur to reduce noise
        blurred = cv.GaussianBlur(gray, (5, 5), 0)

        # Binary threshold
        # (Use THRESH_BINARY_INV if object is darker)
        _, binary = cv.threshold(blurred, 128, 255, cv.THRESH_BINARY_INV)

        # Optional: clean up noise with morphological opening
        kernel = np.ones((3, 3), np.uint8)
        clean = cv.morphologyEx(binary, cv.MORPH_OPEN, kernel)

        # Find contours
        contours, _ = cv.findContours(clean, cv.RETR_EXTERNAL, cv.CHAIN_APPROX_NONE)

        return contours


## **Step 2 – Detecting the Aruco marker** 

OpenCV includes several predefined dictionaries of ArUco markers, each with a different number of markers and sizes. These dictionaries help in generating and detecting markers efficiently. ArUco markers are detected by identifying their square shape and decoding the binary pattern inside. This allows for robust detection even in varying lighting conditions and angles. This makes them perfect for this lab where we are going to use them as know objects to measure unknow object using something called pixel to cm ratio. 

Create a new “main” script that can call your class MyDetectionMethods.  Now create a live stream from your camera that can detect and draw a box around the aruco-marker (if you don’t have movable camera you can record a movie with your phone and use that as a stream instead) you have been given on canvas, you might need to print one.    If correctly programed, you should get a green rectangle around the aruco-marker.

In [13]:
import cv2 as cv
import cv2.aruco as aruco
import numpy as np

# ------------------------
# Video source
# ------------------------
cap = cv.VideoCapture(0)  # 0 = default camera

# ------------------------
# ArUco dictionary and parameters
# ------------------------
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
try:
    parameters = aruco.DetectorParameters_create()
except AttributeError:
    try:
        parameters = aruco.DetectorParameters()
    except Exception:
        parameters = None

# ------------------------
# Create window + zoom trackbar
# ------------------------
cv.namedWindow("ArUco Detection")
cv.createTrackbar("Zoom (%)", "ArUco Detection", 100, 300, lambda x: None)

# ------------------------
# Main loop
# ------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    h, w = frame.shape[:2]

    # Read zoom factor
    zoom_percent = cv.getTrackbarPos("Zoom (%)", "ArUco Detection")
    zoom_percent = max(100, zoom_percent)  # avoid 0%
    zoom_factor = zoom_percent / 100.0

    # Compute crop coordinates (center crop)
    crop_w = int(w / zoom_factor)
    crop_h = int(h / zoom_factor)
    x1 = (w - crop_w) // 2
    y1 = (h - crop_h) // 2
    x2 = x1 + crop_w
    y2 = y1 + crop_h

    # Crop and resize back to original window size
    cropped = frame[y1:y2, x1:x2]
    frame_zoom = cv.resize(cropped, (w, h), interpolation=cv.INTER_LINEAR)

    gray = cv.cvtColor(frame_zoom, cv.COLOR_BGR2GRAY)

    # ------------------------
    # Detect ArUco markers
    # ------------------------
    try:
        if hasattr(aruco, 'detectMarkers'):
            corners, ids, _ = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
        elif hasattr(aruco, 'ArucoDetector'):
            if parameters is not None:
                detector_obj = aruco.ArucoDetector(aruco_dict, parameters)
            else:
                detector_obj = aruco.ArucoDetector(aruco_dict)
            corners, ids, _ = detector_obj.detectMarkers(gray)
        else:
            corners, ids, _ = cv.aruco.detectMarkers(gray, aruco_dict, parameters=parameters)
    except Exception:
        corners, ids = [], None

    if ids is not None:
        for corner in corners:
            corner = corner.reshape(4, 2).astype(int)
            cv.polylines(frame_zoom, [corner], isClosed=True, color=(0, 255, 0), thickness=2)
        for i, corner in enumerate(corners):
            c = corner.reshape(4, 2)
            cv.putText(frame_zoom, f"ID:{ids[i][0]}", tuple(c[0].astype(int)), cv.FONT_HERSHEY_SIMPLEX,
                       0.7, (0, 255, 0), 2)

    # ------------------------
    # Detect other objects & plot centroid
    # ------------------------
    detector = MyDetectionMethods()
    contours = detector.get_contours_canny(frame_zoom)
    for cnt in contours:
        cv.drawContours(frame_zoom, [cnt], -1, (255, 0, 0), 2)
        M = cv.moments(cnt)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            cv.circle(frame_zoom, (cx, cy), 5, (0, 0, 255), -1)
            cv.putText(frame_zoom, f"({cx},{cy})", (cx + 10, cy - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 0, 255), 1)

    # Display
    cv.imshow("ArUco Detection", frame_zoom)

    if cv.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv.destroyAllWindows()


error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\highgui\src\window_w32.cpp:2570: error: (-27:Null pointer) NULL window: 'ArUco Detection' in function 'cvGetTrackbarPos'


## **Step 3 – Object Measurement** 
Place an object beside the aruco-marker. Make the program draw a box around the detected object and give information regarding the length and height in the window, such as this:

You will now need to combine the output from MyDetectionMethods (which should be countour object) with the pixel to cm ratio calculated from the aruco marker to make a proper measurement.  


In [2]:
import cv2 as cv
import cv2.aruco as aruco
import numpy as np
from datetime import datetime


# ------------------------
# Video source
# ------------------------
cap = cv.VideoCapture(0)

# ------------------------
# ArUco dictionary and parameters
# ------------------------
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
try:
    parameters = aruco.DetectorParameters_create()
except AttributeError:
    parameters = aruco.DetectorParameters()

# Use ArucoDetector if available (OpenCV >=4.7)
try:
    detector_obj = aruco.ArucoDetector(aruco_dict, parameters)
    def detect_markers(gray_img):
        return detector_obj.detectMarkers(gray_img)
except AttributeError:
    def detect_markers(gray_img):
        return aruco.detectMarkers(gray_img, aruco_dict, parameters=parameters)

# ------------------------
# Known marker size (cm)
# ------------------------
marker_size_cm = 5.0

# ------------------------
# Detection class
# ------------------------
detector = MyDetectionMethods()

# ------------------------
# Main loop
# ------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        print("Failed to grab frame")
        break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # ------------------------
    # Detect ArUco marker
    # ------------------------
    corners, ids, _ = detect_markers(gray)
    pixel_per_cm = None

    if ids is not None and len(corners) > 0:
        c = corners[0].reshape(4,2)
        cv.polylines(frame, [c.astype(int)], isClosed=True, color=(0,255,0), thickness=2)

        width_px = np.linalg.norm(c[0] - c[1])
        height_px = np.linalg.norm(c[1] - c[2])
        pixel_per_cm = (width_px + height_px) / (2 * marker_size_cm)

        cv.putText(frame, f"Marker ID:{ids[0][0]}", tuple(c[0].astype(int)),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,0), 2)

    # ------------------------
    # Detect objects
    # ------------------------
    contours = detector.get_contours_canny(frame)

    for cnt in contours:
        if cv.contourArea(cnt) < 100:
            continue

        x, y, w, h = cv.boundingRect(cnt)
        cv.rectangle(frame, (x, y), (x+w, y+h), (255,0,0), 2)

        if pixel_per_cm is not None:
            width_cm = w / pixel_per_cm
            height_cm = h / pixel_per_cm
            cv.putText(frame, f"{width_cm:.1f}cm x {height_cm:.1f}cm",
                       (x, y-10), cv.FONT_HERSHEY_SIMPLEX, 0.6, (255,0,0), 2)

        # Centroid
        M = cv.moments(cnt)
        if M["m00"] != 0:
            cx = int(M["m10"]/M["m00"])
            cy = int(M["m01"]/M["m00"])
            cv.circle(frame, (cx, cy), 5, (0,0,255), -1)
            cv.putText(frame, f"({cx},{cy})", (cx+10, cy-10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,255), 1)

    # ------------------------
    # Display date
    # ------------------------
    date_str = datetime.now().strftime("Machine Vision Date: %Y-%m-%d")
    cv.putText(frame, date_str, (10,30), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0,255,255), 2)

    # ------------------------
    # Show frame
    # ------------------------
    cv.imshow("Object Measurement", frame)

    if cv.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv.destroyAllWindows()


## **Step 4 – Specific Object Measurement** 
Place multiple objects at the same time in front of the camera. The program should be able to detect objects approximately of credit card size (85.6 mm wide by 53.9 mm  high) and of an AAA battery cell size (10.5 mm in diameter and 44.5 mm in length, including the positive terminal button) with +/- 5 mm precision. All other object needs to be ignored by the program.  

In [7]:
import cv2 as cv
import cv2.aruco as aruco
import numpy as np
from datetime import datetime


# ------------------------
# Video source
# ------------------------
cap = cv.VideoCapture(0)  # use 0 for default camera, or path to video file

# ------------------------
# ArUco dictionary
# ------------------------
aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)

# Robust detector parameters for all OpenCV versions
try:
    parameters = aruco.DetectorParameters_create()  # older versions
except AttributeError:
    try:
        parameters = aruco.DetectorParameters()     # newer versions
    except Exception:
        parameters = None

# ------------------------
# Known physical size of marker (cm)
# ------------------------
marker_size_cm = 5.0  # e.g., 5 cm marker

# ------------------------
# Expected object sizes (cm)
# ------------------------
credit_card = (8.56, 5.39)   # width x height
aaa_battery = (1.05, 4.45)   # width x height
tolerance_cm = 0.5           # +/- 0.5 cm tolerance

# ------------------------
# MyDetectionMethods instance
# ------------------------
detector = MyDetectionMethods()

# ------------------------
# Main loop
# ------------------------
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv.cvtColor(frame, cv.COLOR_BGR2GRAY)

    # ------------------------
    # Detect ArUco marker
    # ------------------------
    corners, ids, _ = [], None, None
    if hasattr(aruco, 'ArucoDetector'):  # new API
        if parameters is not None:
            detector_obj = aruco.ArucoDetector(aruco_dict, parameters)
        else:
            detector_obj = aruco.ArucoDetector(aruco_dict)
        corners, ids, _ = detector_obj.detectMarkers(gray)
    else:
        corners, ids, _ = aruco.detectMarkers(gray, aruco_dict, parameters=parameters)

    pixel_per_cm = None
    if ids is not None and len(corners) > 0:
        c = corners[0].reshape(4, 2)
        cv.polylines(frame, [c.astype(int)], isClosed=True, color=(0, 255, 0), thickness=2)

        # Compute average pixel size of marker (width & height)
        width_px = np.linalg.norm(c[0] - c[1])
        height_px = np.linalg.norm(c[1] - c[2])
        pixel_per_cm = (width_px + height_px) / (2 * marker_size_cm)

        cv.putText(frame, f"Marker ID:{ids[0][0]}", tuple(c[0].astype(int)),
                   cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 0), 2)

    # ------------------------
    # Detect objects using MyDetectionMethods
    # ------------------------
    contours = detector.get_contours_canny(frame)

    for cnt in contours:
        if cv.contourArea(cnt) < 100:  # ignore tiny noise
            continue

        x, y, w, h = cv.boundingRect(cnt)

        if pixel_per_cm is not None:
            width_cm = w / pixel_per_cm
            height_cm = h / pixel_per_cm

            # Match Credit Card
            if (abs(width_cm - credit_card[0]) <= tolerance_cm and
                abs(height_cm - credit_card[1]) <= tolerance_cm):
                label = "Credit Card"
                color = (255, 0, 0)  # blue

            # Match AAA Battery
            elif (abs(width_cm - aaa_battery[0]) <= tolerance_cm and
                  abs(height_cm - aaa_battery[1]) <= tolerance_cm):
                label = "AAA Battery"
                color = (0, 0, 255)  # red

            else:
                continue  # ignore other objects

            # Draw bounding box
            cv.rectangle(frame, (x, y), (x + w, y + h), color, 2)
            cv.putText(frame, f"{label} ({width_cm:.1f}x{height_cm:.1f} cm)",
                       (x, y - 10), cv.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Draw centroid
        M = cv.moments(cnt)
        if M["m00"] != 0:
            cx = int(M["m10"] / M["m00"])
            cy = int(M["m01"] / M["m00"])
            cv.circle(frame, (cx, cy), 5, (0, 255, 255), -1)
            cv.putText(frame, f"({cx},{cy})", (cx + 10, cy - 10),
                       cv.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 255), 1)

    # ------------------------
    # Display date
    # ------------------------
    date_str = datetime.now().strftime("Machine Vision Date: %Y-%m-%d")
    cv.putText(frame, date_str, (10, 30), cv.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)

    # ------------------------
    # Show result
    # ------------------------
    cv.imshow("Specific Object Measurement", frame)
    if cv.waitKey(1) & 0xFF == 27:  # ESC key
        break

cap.release()
cv.destroyAllWindows()
